## interactions

Update cchen.repo_diversity_interactions table for Superset dashboard monthly

In [4]:
import time
import datetime

import pandas as pd
import requests
from wmfdata import hive
import os

In [5]:
last_month = datetime.date.today().replace(day=1) - datetime.timedelta(days=1)

METRICS_MONTH_TEXT = last_month.strftime("%Y-%m")
metrics_month = pd.Period(METRICS_MONTH_TEXT)

In [6]:
metrics_month

Period('2021-07', 'M')

In [ ]:
## add interactions
hive.run(
"""
insert into cchen.repo_diversity_interactions 

SELECT  month, project, project_family, agent_type, market, access_method, sum(interactions) as interactions 
FROM
(
    SELECT 
            CONCAT(year,'-',LPAD(month,2,'0'),'-01 00:00:00.0') AS month,
            'pageviews' AS type,
            project, 
            canonical.database_group AS project_family,
            agent_type,
            countries.economic_region AS market,
            access_method, 
            SUM(view_count) AS interactions
        FROM wmf.pageview_hourly pv
            LEFT JOIN canonical_data.countries AS countries ON pv.country_code = countries.iso_code
            LEFT JOIN canonical_data.wikis AS canonical ON CONCAT(pv.project,'.org') = canonical.domain_name
       WHERE (year = {year} and month = {month} )
            AND agent_type != 'spider'
            AND NOT (country_code IN ('PK', 'IR', 'AF') 
            AND user_agent_map['browser_family'] = 'IE') 
        GROUP BY CONCAT(year,'-',LPAD(month,2,'0'),'-01 00:00:00.0'),
            canonical.database_group, 
            project,
            agent_type,
            access_method,
            countries.economic_region
    union all
    SELECT  
            CONCAT(year,'-',LPAD(month,2,'0'),'-01 00:00:00.0') AS month, 
            'previews' AS type, 
             project,  
             canonical.database_group AS project_family, 
             agent_type, 
             countries.economic_region AS market, 
             access_method,  
             SUM(view_count) AS interactions 
        FROM wmf.virtualpageview_hourly pv  
            LEFT JOIN canonical_data.countries AS countries ON pv.country_code = countries.iso_code 
            LEFT JOIN canonical_data.wikis AS canonical ON CONCAT(pv.project,'.org') = canonical.domain_name 
        WHERE (year = {year} and month = {month} )
            AND agent_type != 'spider' 
        GROUP BY CONCAT(year,'-',LPAD(month,2,'0'),'-01 00:00:00.0'),
                canonical.database_group, 
                project,
                agent_type,
                access_method,
                countries.economic_region
) x
group by  month, project, project_family, agent_type, market, access_method

""".format(
        year = metrics_month.year,
        month = metrics_month.month
    ))

In [ ]:
#update permission 
os.system("hdfs dfs -chmod -R o+r /user/hive/warehouse/cchen.db/repo_diversity_interactions")